In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# %matplotlib inline

In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available", len(physical_devices))

Num GPUs Available 1


In [3]:

directory = "./FACD_image"

classes = ["1977", "Amaro", "Apollo", "Brannan", "Earlybird"]
target_size = (224,224)
seed = 42;


train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input, validation_split=0.2) \
    .flow_from_directory(directory=directory, target_size=target_size, classes=classes, batch_size=10, subset='training')
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input, validation_split=0.2) \
    .flow_from_directory(directory=directory, target_size=target_size, classes=classes, batch_size=10, subset='validation')
#test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
#    .flow_from_directory(directory=directory, target_size=target_size, classes=classes, batch_size=10, shuffle=False)
        

Found 5120 images belonging to 5 classes.
Found 1280 images belonging to 5 classes.


In [4]:
model = Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(224,224,3)), 
    MaxPool2D(pool_size=(2, 2), strides=2), 
    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'), 
    MaxPool2D(pool_size=(2,2), strides=2), 
    Flatten(), 
    Dense(units=len(classes), activation='softmax'),
])

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 112, 112, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 200704)            0         
_________________________________________________________________
dense (Dense)                (None, 5)                 1003525   
Total params: 1,022,917
Trainable params: 1,022,917
Non-trainable params: 0
______________________________________________

In [6]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
model.fit(x=train_batches, validation_data=valid_batches, epochs=10, verbose=1)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 512 steps, validate for 128 steps
Epoch 1/10
512/512 [==============================] - 102s 199ms/step - loss: 7.8989 - accuracy: 0.6715 - val_loss: 0.8480 - val_accuracy: 0.8477
Epoch 2/10
512/512 [==============================] - 95s 186ms/step - loss: 0.5615 - accuracy: 0.8709 - val_loss: 0.6328 - val_accuracy: 0.8562
Epoch 3/10
512/512 [==============================] - 94s 183ms/step - loss: 0.2804 - accuracy: 0.9199 - val_loss: 0.6117 - val_accuracy: 0.8664
Epoch 4/10
512/512 [==============================] - 95s 185ms/step - loss: 0.2231 - accuracy: 0.9361 - val_loss: 0.5703 - val_accuracy: 0.8805
Epoch 5/10
512/512 [==============================] - 94s 185ms/step - loss: 0.1813 - accuracy: 0.9492 - val_loss: 0.4844 - val_accuracy: 0.8859
Epoch 6/10
512/512 [==============================] - 99s 193ms/step - loss: 0.1293 - accuracy: 0.9604 - val_loss: 0.6309 - val_accuracy: 0.8703
Epoch 7/10
512/512 [===============

In [8]:
filepath = "./keras_five_filters_color_few_layers.hdf5"

model.save(
   filepath, overwrite=True, include_optimizer=True, save_format="h5",
    signatures=None, options=None
)